## Chat with the SuperStore Dataset

In [9]:
#importing the necessary dependencies
import os
from langchain.llms import GooglePalm
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
#loading g the llm
llm=GooglePalm(google_api_key=os.environ['GOOGLE_API_KEY'],temperature=0.6)

c:\Users\ADMIN\Documents\Data Analytics\Machine Learning\NLP\NLP Projects\Text_to_SQL_SuperStore\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
#connecting to SQL database:Superstore database
from langchain.utilities import SQLDatabase

#db_password: Do Not Use Special Characters
db_user = "root"
db_password = "root1234"
db_host = "localhost"
db_name = "sample_superstore"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE orders (
	`Row_ID` INTEGER, 
	`Order_ID` TEXT, 
	`Order_Date` TEXT, 
	`Ship_Date` TEXT, 
	`Ship_Mode` TEXT, 
	`Customer_ID` TEXT, 
	`Customer_Name` TEXT, 
	`Segment` TEXT, 
	`Country` TEXT, 
	`City` TEXT, 
	`State` TEXT, 
	`Postal_Code` TEXT, 
	`Region` TEXT, 
	`Product_ID` TEXT, 
	`Category` TEXT, 
	`Sub_Category` TEXT, 
	`Product_Name` TEXT, 
	`Sales` DOUBLE, 
	`Quantity` INTEGER, 
	`Discount` DOUBLE, 
	`Profit` DOUBLE
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from orders table:
Row_ID	Order_ID	Order_Date	Ship_Date	Ship_Mode	Customer_ID	Customer_Name	Segment	Country	City	State	Postal_Code	Region	Product_ID	Category	Sub_Category	Product_Name	Sales	Quantity	Discount	Profit
1	CA-2020-152156	11/8/2020	11/11/2020	Second Class	CG-12520	Claire Gute	Consumer	United States	Henderson	Kentucky	42420	South	FUR-BO-10001798	Furniture	Bookcases	Bush Somerset Collection Bookcase	261.9600000000	2	0E-10	41.9136000000
2	CA-2020-152156	11/8/2020	11/11/202

In [22]:
#using SQL chain
from langchain_experimental.sql import SQLDatabaseChain

#Put verbose=true to view the generated SQL statements
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True)

### Exploring the Answers without adding Few Shot Learning

In [23]:
qns1 = db_chain.run("what is the total number of returns made by the store")



> Entering new SQLDatabaseChain chain...
what is the total number of returns made by the store
SQLQuery:SELECT COUNT(*) FROM returns
SQLResult: [(800,)]
Answer:800
> Finished chain.


Need to be more specific to differentiate between the number of products returned and the number of orders returned. Let's check if it can tell the difference.

In [24]:
qns1 = db_chain.run("what is the total number of orders returned to the store")



> Entering new SQLDatabaseChain chain...
what is the total number of orders returned to the store
SQLQuery:SELECT COUNT(*) FROM returns
SQLResult: [(800,)]
Answer:800
> Finished chain.


In [25]:
qns1 = db_chain.run("what is the total number of products returned to the store")



> Entering new SQLDatabaseChain chain...
what is the total number of products returned to the store
SQLQuery:SELECT COUNT(*) AS Total_Products_Returned FROM returns
SQLResult: [(800,)]
Answer:800
> Finished chain.


Seems it can't tell the difference. This is an example of a query that should be used in few shot learning

In [26]:
qns1 = db_chain.run("what is the total sales made by the store")



> Entering new SQLDatabaseChain chain...
what is the total sales made by the store
SQLQuery:SELECT SUM(Sales) FROM orders
SQLResult: [(2272449.8562999545,)]
Answer:2272449.8562999545
> Finished chain.
